In [18]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from tqdm import tqdm

In [56]:
#데이터 프레임을 읽어오기
chat_df = pd.read_csv('LINE_data.csv')

In [64]:
def refine_conversation(df):
    # 대화를 저장할 리스트 초기화
    conversations = []
    current_conversation = []  # 현재 대화를 저장할 리스트 초기화
    last_message_time = None  # 마지막 메시지 시간을 추적하기 위한 변수 초기화

    # 데이터프레임의 각 행을 반복 처리, 진행 상황을 표시하기 위해 tqdm 사용
    for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="Processing Conversations"):
        if row['Sender'] != 'LSN':
            # 마지막 메시지 시간과 현재 메시지 시간의 차이가 1시간을 초과하면 대화를 종료
            if current_conversation and (row['DateTime'] - last_message_time > timedelta(hours=1)):
                # 현재 대화를 'Prev_Message'와 'My_Response'로 분리하여 conversations 리스트에 추가
                prev_message = ' '.join([msg['Message'] for msg in current_conversation if msg['Sender'] != 'LSN'])
                my_response = ' '.join([msg['Message'] for msg in current_conversation if msg['Sender'] == 'LSN'])
                if my_response.strip():  # My_Response가 공백이 아닌 경우에만 추가
                    conversations.append({
                        'Prev_Message': prev_message,
                        'My_Response': my_response
                    })
                # 현재 대화 초기화
                current_conversation = []

            # 현재 메시지를 현재 대화 리스트에 추가
            current_conversation.append(row)
            # 마지막 메시지 시간 업데이트
            last_message_time = row['DateTime']
            
        elif row['Sender'] == 'LSN':
            if current_conversation:
                # LSN의 응답을 현재 대화에 추가하고 대화를 종료
                current_conversation.append(row)
                prev_message = ' '.join([msg['Message'] for msg in current_conversation if msg['Sender'] != 'LSN'])
                my_response = ' '.join([msg['Message'] for msg in current_conversation if msg['Sender'] == 'LSN'])
                if my_response.strip():  # My_Response가 공백이 아닌 경우에만 추가
                    conversations.append({
                        'Prev_Message': prev_message,
                        'My_Response': my_response
                    })
                # 현재 대화 초기화
                current_conversation = []
            # 마지막 메시지 시간 업데이트
            last_message_time = row['DateTime']

    # 마지막 대화가 남아 있을 경우 추가
    if current_conversation:
        prev_message = ' '.join([msg['Message'] for msg in current_conversation if msg['Sender'] != 'LSN'])
        my_response = ' '.join([msg['Message'] for msg in current_conversation if msg['Sender'] == 'LSN'])
        if my_response.strip():  # My_Response가 공백이 아닌 경우에만 추가
            conversations.append({
                'Prev_Message': prev_message,
                'My_Response': my_response
            })

    # 대화 목록을 데이터프레임으로 반환
    return pd.DataFrame(conversations)

In [70]:
def preprocess_chat_data(df):
    df['DateTime'] = pd.to_datetime(df['DateTime'], format='%Y-%m-%d %H:%M:%S')
    result_df = refine_conversation(df)
    return result_df

In [66]:
result_df = preprocess_chat_data(chat_df)

Processing Conversations: 100%|██████████| 474706/474706 [00:19<00:00, 24378.68it/s]


In [73]:
result_df.to_csv('preprocessing_df.csv', index=False)